In [2]:
from scipy.special import factorial
import numpy as np
from scipy.stats import chi2
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from scipy.stats import poisson
from scipy.stats import chisquare
import plotly.graph_objects as go
from metropolis_hastings import metropolis_hastings, truncated_poisson_samples,metropolis_hastings_joint,y_sampling_function

# Exercise 6

The number of busy lines in a trunk group (Erlang system) is
given by a truncated Poisson distribution
P(i) = c ·
Ai
i!
, i = 0, . . . m
Generate values from this distribution by applying the
Metropolis-Hastings algorithm, verify with a χ
2
-test. You can
use the parameter values from exercise 4

In [3]:
m = 10 #number of servers
s = 8 #mean service time
lam = 1#arrival_intensity
N =100000 #10*10000
A = lam*s


In [4]:
#1The number of busy lines in a trunk group (Erlang system) is
#given by a truncated Poisson distribution

#unnormalized probability mass function
g = lambda x : A**x / factorial(x)

X_est = metropolis_hastings(g,N,m)
#X_est is the number of busy lines in a trunk group
#create histogram and plot i
hist_estimate,_ = np.histogram(X_est, bins = np.arange(m+2))

#sample from the truncated poisson distribution
X_expected = truncated_poisson_samples(A, 0, m, N)
hist_expected,_ = np.histogram(X_expected, bins = np.arange(m+2))

#Do a Plotly histogram of the estimated values and the expected values in the same plot
fig = go.Figure()
fig.add_trace(go.Bar(x = list(range(m+1)), y = hist_estimate, name = 'Estimated'))
fig.add_trace(go.Bar(x = list(range(m+1)), y = hist_expected, name = 'Expected'))
fig.update_layout(title = 'Estimated and Expected values of the number of busy lines in a trunk group',
                  xaxis_title = 'Number of busy lines',
                  yaxis_title = 'Count')
fig.show()

T, p_value = chisquare(hist_estimate/N, hist_expected/N)
print('Chi-square test statistic:', T)
print('P-value:', p_value)

Chi-square test statistic: 0.0002173359618935817
P-value: 1.0


### 2
For two different call types the joint number of occupied lines
is given by ... 
You can use A1, A2 = 4 and m = 10

#### a)
Use Metropolis-Hastings, directly to generate variates from
this distribution.

In [5]:
A1 = 4
A2 = 4

def g_joint(A1,A2):
    return lambda x,y : A1**x / factorial(x) * A2**(y) / factorial(y)

def joint_truncated_poisson_samples(A1, A2, low, high, size=1):
    samples = []
    while len(samples) < size:
        x = np.random.poisson(A1)
        y = np.random.poisson(A2)
        if low <= x + y <= high:
            samples.append((x,y))
    return np.array(samples)

In [7]:
X_est_joint = metropolis_hastings_joint(g_joint(A1,A2),N, y_sampling_func=y_sampling_function(m))
X_true_joint = joint_truncated_poisson_samples(A1,A2,0,m,N)

#joint histogram estimate
hist_joint_estimate,_,_ = np.histogram2d(X_est_joint[:,0],X_est_joint[:,1], bins = [np.arange(m+2),np.arange(m+2)])
#joint histogram true
hist_joint_true,_,_ = np.histogram2d(X_true_joint[:,0],X_true_joint[:,1], bins = [np.arange(m+2),np.arange(m+2)])
#plot the joint estimated joint histogram
fig = go.Figure(data = [go.Heatmap(z = hist_joint_estimate)])
fig.update_layout(title = 'Estimated joint histogram of the number of busy lines in a trunk group',
                  xaxis_title = 'Number of busy lines in trunk 1',
                  yaxis_title = 'Number of busy lines in trunk 2')
fig.show()
#plot the joint true joint histogram
fig = go.Figure(data = [go.Heatmap(z = hist_joint_true)])
fig.update_layout(title = 'True joint histogram of the number of busy lines in a trunk group',
                  xaxis_title = 'Number of busy lines in trunk 1',
                  yaxis_title = 'Number of busy lines in trunk 2')
fig.show()

In [8]:
#calculate the test statistic for all histograms > 0
T, p_value = chisquare(hist_joint_estimate.flatten()[hist_joint_estimate.flatten() > 0]/N, hist_joint_true.flatten()[hist_joint_estimate.flatten() > 0]/N)
print('Chi-square test statistic:', T)
print('P-value:', p_value)

Chi-square test statistic: 0.03653932542368645
P-value: 1.0


### c
Use Gibbs sampling to sample from the distribution. This is
(also) coordinate-wise but here we use the exact conditional
distributions. You will need to find the conditional
distributions analytically

In [9]:
# TODO

### 3
 We consider a Bayesian statistical problem. The observations
are Xi ∼ N(Θ, Ψ), where the prior distribution of the pair
(Ξ, Γ) = (log (Θ), log (Ψ)) is standard normal with correlation
ρ. The posterior distribution of (Θ, Ψ) is given by...
which can be derived using a standard change of variable
technique. The task of this exercise is now to sample from the
posterior distribution of (Θ, Ψ) using Markov Chain Monte
Carlo.

#### a)
 Generate a pair (θ, ψ) from the prior distribution, i.e. the
distribution for the pair (Θ, Ψ), by first generating a sample
(ξ, γ) of (Ξ, Γ).


In [162]:
#xi and gamma of (Ξ, Γ) = (log (Θ), log (Ψ)) 
#joint density of Theta and Psi is normal with correlation rho = 0.5
rho = 0.5

#sample from the joint density
θ, ψ =np.exp(np.random.multivariate_normal([0,0],[[1,rho],[rho,1]]))

print('θ:',θ)
print('ψ:',ψ)

θ: 1.1249104643519519
ψ: 1.6218629641819389


#### b)
 Generate Xi = 1, . . . , n with the values of (θ, ψ) you
obtained in item 3a. Use n = 10

In [181]:
n = 100
#generate n samples from the univariate distribution N(θ,ψ)
np.random.seed(100)
X = np.random.normal(θ,ψ,n)

In [163]:

#The log-likelihood of the data X given the parameters θ and ψ is. Each X follows Normal(θ,ψ)
likelihood = lambda X,θ,ψ : np.exp(-n/2*np.log(2*np.pi) - n/2*np.log(ψ**2) - 1/(2*ψ**2+ 1e-16)*np.sum((X-θ)**2))
#  The joint density f(x, y) of (Θ, Ψ) is
prior = lambda  x,y : 1/(2*np.pi*x*y*np.sqrt(1-rho**2))*np.exp(-1/(2*(1-rho**2))*(np.log(x)**2 - 2*rho*np.log(x)*np.log(y) + np.log(y)**2))

#y is sampled from a folded normal distribution
y_sampling_bays = lambda : np.abs(np.random.normal(0,4,2))

posterior = lambda X : lambda θ,ψ : likelihood(X,θ,ψ)*prior(θ,ψ)
likelihood(X,θ,ψ)

2.710291520790959e-82

In [165]:
X_est_joint = metropolis_hastings_joint(posterior(X),N,m, y_sampling_func= y_sampling_bays) 

In [178]:
#joint histogram of the X_est_joint samples in the interval \[θ-1,θ+1\] and \[ψ-1,ψ+1\]
X_est_joint_interval = X_est_joint[(X_est_joint[:,0] > θ-1)*( X_est_joint[:,0] < θ+1)*(X_est_joint[:,1] > ψ-1)*( X_est_joint[:,1] < ψ+1)]
hist_joint_estimate,_,_ = np.histogram2d(X_est_joint_interval[:,0],X_est_joint_interval[:,1], bins = [20,20])
#Plotly histogram of the joint histogram along with the true values of θ and ψ
fig = go.Figure(data = [go.Heatmap(z = hist_joint_estimate, x = np.linspace(θ-1,θ+1,20), y = np.linspace(ψ-1,ψ+1,20), name = 'Estimated')])
fig.add_trace(go.Scatter(x = [θ], y = [ψ], mode = 'markers', marker = dict(size = 10, color = 'green'), name = 'true θ and ψ'))
fig.update_layout(title = 'Estimated joint histogram of θ and ψ',
                  xaxis_title = 'θ',
                  yaxis_title = 'ψ',
                    showlegend = True,
                    legend = dict(x = 0.8, y = 0.8))

fig.show()